In [3]:
! pip install simpletransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 14.2

In [4]:
!pip install wandb

In [6]:
import json
with open(r"train.json","r") as read_file:
  train=json.load(read_file)

In [7]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?',
    'answers': [{'text': 'Brandon Sanderson', 'answer_start': 71}]}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?',
    'answers': [{'text': 'between 2006 and 2008', 'answer_start': 28}]},
   {'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?',
    'answers': [{'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages',
      'answer_start': 63}]},
   {'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?',
    'answers': []}]}]

In [8]:
with open(r"test.json","r") as read_file:
  test=json.load(read_file)

In [9]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?',
    'answers': [{'text': 'region called the Final Empire', 'answer_start': 38},
     {'text': 'world called Scadrial', 'answer_start': 74}]}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?',
    'answers': [{'text': 'one', 'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?',
    'answers': []}]}]

In [10]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [11]:
model_type="bert"
model_name ="bert-base-cased"
if model_type=="bert":
  model_name="bert-base-cased"
elif model_type=="roberta":
  model_name="roberta_base"
elif model_type=="distilbert":
  model_name="distilbert-base-cased"
elif model_type=="distilroberta":
  model_type="roberta"
  model_name="distilroberta-base"
elif model_type=="electra-base":
  model_type="electra"
  model_name="google/electra-base-discriminator"
elif model_type=="electra-small":
  model_type="electra"
  model_name="google/electra-small-discriminator"
elif model_type=="xlnet":
  model_name="xlnet-base-cased"



In [12]:
# configure the model QuestionAnsweringArgs help us configure the model
model_args= QuestionAnsweringArgs()
model_args.train_batch_size=16
model_args.evaluate_during_training=True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [19]:
### advanced methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features":True,
    "output_dir":f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs":10,
    "evaluate_during_training_steps": 1000,
    "wandb_project":"Question Answer Application",
    "wandb_kwargs":{"name": model_name}, # is used for visualization of the training
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "train_batch_size":128,
    "eval_batch_size":64,


}

In [20]:
# next use the question answering model
model=QuestionAnsweringModel(
    model_type, model_name,args=train_args, use_cuda=False
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# if there is an output folder and you want to retrain the model run the line else don't run the cell
!rm -rf outputs


In [21]:
# train the model
model.train_model(train,eval_data=test)

add example index and unique id: 100%|██████████| 4/4 [00:00<00:00, 6388.89it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

correct,█▁▁▁▁
eval_loss,█▅▃▂▁
global_step,▁▃▅▆█
incorrect,▁▁▁▁▁
similar,▁████
train_loss,██▅▃▁
correct,0
eval_loss,-0.10574
global_step,5
incorrect,2
similar,1


Running Epoch 1 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(10,
 {'global_step': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'correct': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'similar': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2],
  'incorrect': [2, 2, 2, 2, 2, 2, 1, 1, 1, 1],
  'train_loss': [4.861111164093018,
   4.901000022888184,
   4.375502586364746,
   3.8710274696350098,
   3.4425477981567383,
   3.0857889652252197,
   2.5783064365386963,
   2.4342522621154785,
   2.0674710273742676,
   2.0257320404052734],
  'eval_loss': [0.0439932681620121,
   -0.04081438109278679,
   -0.1382858008146286,
   -0.2512153387069702,
   -0.3808571994304657,
   -0.5146710872650146,
   -0.6347161531448364,
   -0.7305894494056702,
   -0.797112226486206,
   -0.830558717250824]})

In [22]:
# evaluate the model
result, text = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
result

{'correct': 0, 'similar': 2, 'incorrect': 1, 'eval_loss': -0.830558717250824}

In [24]:
text

{'correct_text': {},
 'similar_text': {'00001': {'truth': 'region called the Final Empire',
   'predicted': '',
   'question': 'Where does the series take place?'},
  '00003': {'truth': '',
   'predicted': 'omantic powers, while "',
   'question': 'What are Allomantic powers?'}},
 'incorrect_text': {'00002': {'truth': 'one',
   'predicted': 'omantic powers, while "',
   'question': 'How many powers does a Misting possess?'}}}

In [25]:
#  make predictions with the model
to_predict=[
    {
        "context":" Vin is a Mistborn of great power and skill",
        "qas":[
            {
                "question": "What is Vin's speciality?",
                "id":"0",}

        ]
    }
]

In [26]:
answers,probabilities=model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3368.92it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Vin', 'Vin is a Mistborn of great power', 'great power']}]
